In [6]:
# 코드에 있는 계산을 여러 CPU 코어가 병렬로 실행할 수 있는 독립적인 작업 단위로 나눈다

# concurrent.futures 내장 모듈의 multiprocessing 내장 모듈은 
# 자식 프로세스로 다른 파이썬 인터프리터를 실행시킴으로써 파이썬에서 여러 CPU 코어를 사용할 수 있게 만들어준다

# 자식 프로세스는 주 인터프리터와 별도로 실행되기 때문에 주 인터프리터의 GIL과 분리된다

# mymodule.py 

# serial 버전, 선형으로 실행시간이 증가한다 ( 병렬 사용 x )

import my_module
import time

NUMBERS = [
    (1963309, 2265973), (2030677, 3814172),
    (1551645, 2229620), (2039045, 2020802),
    (1823712, 1924928), (2293129, 1020491),
    (1281238, 2273782), (3823812, 4237281),
    (3812741, 4729139), (1292391, 2123811),
]

def main():
    start = time.time()
    results = list(map(my_module.gcd, NUMBERS))
    end = time.time()
    delta = end - start
    print(f'총 {delta:.3f}초 걸림')
    
if __name__ == '__main__':
    main()

총 5.897초 걸림


In [10]:
# 코드에 있는 계산을 여러 CPU 코어가 병렬로 실행할 수 있는 독립적인 작업 단위로 나눈다

# concurrent.futures 내장 모듈의 multiprocessing 내장 모듈은 
# 자식 프로세스로 다른 파이썬 인터프리터를 실행시킴으로써 파이썬에서 여러 CPU 코어를 사용할 수 있게 만들어준다

# 자식 프로세스는 주 인터프리터와 별도로 실행되기 때문에 주 인터프리터의 GIL과 분리된다

# mymodule.py

# threads를 run하는 프로그램, 오히려 더 느려짐

import my_module
from concurrent.futures import ThreadPoolExecutor
import time

NUMBERS = [
    (1963309, 2265973), (2030677, 3814172),
    (1551645, 2229620), (2039045, 2020802),
    (1823712, 1924928), (2293129, 1020491),
    (1281238, 2273782), (3823812, 4237281),
    (3812741, 4729139), (1292391, 2123811),
]

def main():
    start = time.time()
    pool = ThreadPoolExecutor(max_workers=2) # 스레드 풀을 시작하고 통신 비용이 들기 때문에 이전보다 더 느려짐
    results = list(pool.map(my_module.gcd, NUMBERS))
    end = time.time()
    delta = end - start
    print(f'총 {delta:.3f}초 걸림')
    
if __name__ == '__main__':
    main()

총 17.526초 걸림


In [15]:
# 코드에 있는 계산을 여러 CPU 코어가 병렬로 실행할 수 있는 독립적인 작업 단위로 나눈다

# concurrent.futures 내장 모듈의 multiprocessing 내장 모듈은 
# 자식 프로세스로 다른 파이썬 인터프리터를 실행시킴으로써 파이썬에서 여러 CPU 코어를 사용할 수 있게 만들어준다

# 자식 프로세스는 주 인터프리터와 별도로 실행되기 때문에 주 인터프리터의 GIL과 분리된다

# mymodule.py

# parallel.. Thread말고 ProcessPool

import my_module
from concurrent.futures import ProcessPoolExecutor
import time

NUMBERS = [
    (1963309, 2265973), (2030677, 3814172),
    (1551645, 2229620), (2039045, 2020802),
    (1823712, 1924928), (2293129, 1020491),
    (1281238, 2273782), (3823812, 4237281),
    (3812741, 4729139), (1292391, 2123811),
]

def main():
    start = time.time()
    pool = ProcessPoolExecutor(max_workers=2) # 스레드 풀을 시작하고 통신 비용이 들기 때문에 이전보다 더 느려짐
    results = list(pool.map(my_module.gcd, NUMBERS))
    end = time.time()
    delta = end - start
    print(f'총 {delta:.3f}초 걸림')
    
if __name__ == '__main__':
    main()

총 5.940초 걸림


1. (부모) 이 객체 (ProcessPoolExecutor 인스턴스)는 입력 데이터로 들어온 map 메서드에 전달된 NUMBERS의 각 원소를 취한다
2. (부모) 이 객체는 1번에서 얻은 원소를 pickle 모듈을 사용해 이진 데이터로 직렬화한다
3. (부모, 자식) 이 객체는 로컬 소켓을 통해 주 인터프리터 프로세스로부터 자식 인터프리터 프로세스에게 2번에서 직렬화 한 데이터를 복사한다.
4. (자식) 이 객체는 pickle을 사용해 데이터를 파이썬 객체로 역직렬화한다.
5. (자식) 이 객체는 gcd 함수가 들어 있는 모듈을 Import 한다
6. (자식) 이 객체는 입력 데이터에 대해 gcd 함수를 실행한다. 이때 다른 자식 인터프리터 프로세스와 병렬로 실행한다
7. (자식) 이 객체는 gcd 함수의 결과를 이진 데이터로 직렬화 한다.
8. (부모, 자식) 이 객체는 로컬 소켓을 통해 자식 인터프리터 프로세스로 부터 부모 인터프리터 프로세스에게 7번에서 직렬화한 결과 데이터를 돌려준다.
9. (부모) 이 객체는 데이터를 파이썬 객체로 역직렬화한다.
10. (부모) 여러 자식 프로세스가 돌려준 결과를 병합해서 한 list로 만든다.